In [2]:
import pandas as pd
import string
from nltk.corpus import stopwords

In [3]:
cd Downloads

/Users/kanth/Downloads


In [6]:
df_spam = pd.read_csv('NB.csv', encoding = 'ISO-8859-1', names =['response','message'])

In [28]:
df_spam.head(10)

,response,message,lenght
0,type,text,4
1,ham,Hope you are having a good week. Just checking in,49
2,ham,K..give back my thanks.,23
3,ham,Am also doing in cbe only. But have to pay.,43
4,spam,"complimentary 4 STAR Ibiza Holiday or å£10,000...",150
5,spam,okmail: Dear Dave this is your final notice to...,161
6,ham,Aiya we discuss later lar... Pick u up at 4 is...,50
7,ham,Are you this much buzy,22
8,ham,Please ask mummy to call father,31
9,spam,Marvel Mobile Play the official Ultimate Spide...,161


In [8]:
df_spam.describe()

,response,message
count,5560,5560
unique,3,5157
top,ham,"Sorry, I'll call later"
freq,4812,30


In [9]:
df_spam.groupby('response').describe()

message                                                            \
           count unique                                                top   
response                                                                     
ham         4812   4503                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   
type           1      1                                               text   

               
         freq  
response       
ham        30  
spam        4  
type        1

In [10]:
df_spam['lenght'] = df_spam['message'].apply(len)

In [11]:
df_spam.head()

,response,message,lenght
0,type,text,4
1,ham,Hope you are having a good week. Just checking in,49
2,ham,K..give back my thanks.,23
3,ham,Am also doing in cbe only. But have to pay.,43
4,spam,"complimentary 4 STAR Ibiza Holiday or å£10,000...",150


In [12]:
def message_text_process(mess):
    no_punctuation = [char for char in mess if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return [word for word in no_punctuation.split() if word.lower() 
            not in stopwords.words('english')]

In [13]:
df_spam['message'].head(5).apply(message_text_process)

0                                               [text]
1                         [Hope, good, week, checking]
2                                [Kgive, back, thanks]
3                                     [also, cbe, pay]
4    [complimentary, 4, STAR, Ibiza, Holiday, å£100...
Name: message, dtype: object

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
bag_words = CountVectorizer(analyzer = message_text_process).fit(df_spam['message'])

In [17]:
print(len(bag_words.vocabulary_))

11356


In [18]:
message_bagwords = bag_words.transform(df_spam['message'])

In [20]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(message_bagwords)

In [21]:
message_tfidf = tfidf_transformer.transform(message_bagwords)

In [30]:
print(message_tfidf.shape)
print(message_tfidf)

(5560, 11356)
  (0, 10317)	1.0
  (1, 10943)	0.4379061279918543
  (1, 6847)	0.4079242323354249
  (1, 5414)	0.6229219165741362
  (1, 2254)	0.5037899657519401
  (2, 10332)	0.5334360757557065
  (2, 4865)	0.39416690547237293
  (2, 2489)	0.7483838612053881
  (3, 8719)	0.5395138634355241
  (3, 5352)	0.6872499784523147
  (3, 4611)	0.4864280607429712
  (4, 11304)	0.28865844724105827
  (4, 8343)	0.2601607148680903
  (4, 7847)	0.2601607148680903
  (4, 5601)	0.2601607148680903
  (4, 5554)	0.23322213622114832
  (4, 5331)	0.20699826547700245
  (4, 4066)	0.21693735065983258
  (4, 3523)	0.32915038778576383
  (4, 2588)	0.2780552476467615
  (4, 2329)	0.29537833715478673
  (4, 2244)	0.24077031610797955
  (4, 1296)	0.32915038778576383
  (4, 605)	0.14754486113546064
  (4, 262)	0.32915038778576383
  :	:
  (5558, 10715)	0.1102127865649493
  (5558, 10637)	0.08307238569114422
  (5558, 10077)	0.2120491324978143
  (5558, 9755)	0.24179460139963893
  (5558, 9120)	0.1353384855415264
  (5558, 8015)	0.170340824177310

In [31]:
from sklearn.naive_bayes import MultinomialNB
spam_detect = MultinomialNB().fit(message_tfidf,df_spam['response'])

In [32]:
predicted = spam_detect.predict(message_tfidf)

In [33]:
predicted

array(['ham', 'ham', 'ham', ..., 'spam', 'spam', 'ham'], dtype='<U4')

In [27]:
expected = df_spam['response']

from sklearn import metrics
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

        ham       0.98      1.00      0.99      4812
       spam       1.00      0.85      0.92       747
       type       0.00      0.00      0.00         1

avg / total       0.98      0.98      0.98      5560

[[4812    0    0]
 [ 115  632    0]
 [   1    0    0]]


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
